# Тестирование

Чтобы запустить тесты, необходимо воспользоваться модулем `unittest` которому указывается файл-тест. Файлы тестов должны запускаться из текущего каталога. Если будет попытка указать файл теста в подкаталоге, то это приведет к исключению `ValueError`:

In [45]:
%%bash

python3 -m unittest ./tests/test_python.py

Traceback (most recent call last):
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/lib/python3.6/unittest/__main__.py", line 18, in <module>
    main(module=None)
  File "/usr/lib/python3.6/unittest/main.py", line 94, in __init__
    self.parseArgs(argv)
  File "/usr/lib/python3.6/unittest/main.py", line 141, in parseArgs
    self.createTests()
  File "/usr/lib/python3.6/unittest/main.py", line 148, in createTests
    self.module)
  File "/usr/lib/python3.6/unittest/loader.py", line 219, in loadTestsFromNames
    suites = [self.loadTestsFromName(name, module) for name in names]
  File "/usr/lib/python3.6/unittest/loader.py", line 219, in <listcomp>
    suites = [self.loadTestsFromName(name, module) for name in names]
  File "/usr/lib/python3.6/unittest/loader.py", line 153, in loadTestsFromName
    module = __import__(module_name)
ValueEr

Установим текущий каталог с нашими тестами и запустим тест `test_python.py`:

In [ ]:
# %load ./tests/test_python.py

import unittest

class TestPython(unittest.TestCase):

    def test_float_to_int_coercion(self):
        self.assertEqual(1, int(1.0))

    def test_get_empty_dict(self):
        self.assertIsNone({}.get('key'))

    def test_trueness(self):
        self.assertTrue(bool(10))


In [46]:
%%bash

cd ./tests/
python3 -m unittest test_python.py

...
----------------------------------------------------------------------
Ran 3 tests in 0.000s

OK


Как мы можем видеть, тест пройден успешно. Точки в самом начале вывода обозначают успешность прохождения каждой последующей функции в файл-тесте. В конце, под чертой, выводится общая информация и общий статус.

Протестируем тест на деление:

In [ ]:
# %load ./tests/test_division.py

import unittest

class TestDivision(unittest.TestCase):
    def test_integer_division(self):
        self.assertIs(10 / 5,2)


In [49]:
%%bash

cd ./tests/
python3 -m unittest test_division.py

F
FAIL: test_integer_division (test_division.TestDivision)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/ubuntu/coursera/4/tests/test_division.py", line 6, in test_integer_division
    self.assertIs(10 / 5,2)
AssertionError: 2.0 is not 2

----------------------------------------------------------------------
Ran 1 test in 0.000s

FAILED (failures=1)


Как мы можем видеть, тест прошел неуспешно, произошло исключение (мы намеренно его создали для демонстрации ошибки).

Сейчас мы напишем класс `Asteriod` который будет выходить в интернет и получать данные астероида 2099942 на сайте NASA.

In [52]:
import requests

class Asteroid():
    BASE_API_URL = 'https://api.nasa.gov/neo/rest/v1/neo/{}?api_key=DEMO_KEY'

    def __init__(self, spk_id):
        self.api_url = self.BASE_API_URL.format(spk_id)
        self._data = None

    def get_data(self):
        if self._data is None:
            self._data = requests.get(self.api_url).json()
            # Выгрузка загруженных данных в файл формата JSON:
            # import json
            # with open('apophis_fixture.txt', 'w') as f:
            #    f.write(json.dumps(self._data))
        return self._data

    @property
    def name(self):
        return self.get_data()['name']

    @property
    def diameter(self):
        return int(self.get_data()['estimated_diameter']['meters']['estimated_diameter_max'])



apophis = Asteroid(2099942)
print('Name: {}, Diameter: {}'.format(apophis.name, apophis.diameter))

Name: 99942 Apophis (2004 MN4), Diameter: 682


Запустим нашу программу из консоли:

In [53]:
%%bash

python3 ./tests/asteroid.py

Name: 99942 Apophis (2004 MN4), Diameter: 682


В данном случае, наш класс выходит в интернет, другому классу возможно потребуется какой-то файл или другой ресурс. 
Что же делать, если на тестовой машине нет интернета или он медленный?  
Напишем тест в файле `test_asteroid.py` для класса `Asteroid` в котором будем использовать механизм mock'ов:

In [ ]:
# %load ./tests/test_asteroid.py
import json
import unittest
from unittest.mock import patch
from asteroid import Asteroid

class TestAsteroid(unittest.TestCase):

    def setUp(self):
        self.asteroid = Asteroid(2099942)

    def moscked_get_data(self):
        # Данные были заранее выкачены и записаны в файл apophis_fixture.txt
        with open('apophis_fixture.txt') as f:
            return json.loads(f.read()) # Из файла считываем данные в объект JSON

    # Здесь идет подмена оригинальной функции get_data на moscked_get_data механизмом Декоратора,
    # которая возвращает данные не из интернета, а из заранее подготовленных данных
    @patch('asteroid.Asteroid.get_data', moscked_get_data)
    def test_name(self):
        self.assertEqual(self.asteroid.name, '99942 Apophis (2004 MN4)')

    # Здесь аналогично предыдущему методу, подменяется оригинальный метод
    @patch('asteroid.Asteroid.get_data', moscked_get_data)
    def test_diameter(self):
        self.assertEqual(self.asteroid.diameter, 682)


Вызовем тест:

In [57]:
%%bash

cd ./tests/
python3 -m unittest test_asteroid.py

Name: 99942 Apophis (2004 MN4), Diameter: 682


..
----------------------------------------------------------------------
Ran 2 tests in 0.005s

OK


Тест успешно пройден.